In [10]:
# !pip install transformers
# !pip install torch

# !pip install simplet5
# !pip install -q chinese-converter
import json
from transformers import LogitsProcessor
from transformers import LogitsProcessorList

# 具体代码
import torch
from simplet5 import SimpleT5
from transformers import T5Tokenizer, T5ForConditionalGeneration
import chinese_converter

MODEL_PATH = "./Models/t5-poem-v2.1/simplet5-epoch-4-train-loss-3.4329-val-loss-3.4315"
class PoemModel(SimpleT5):
  def __init__(self) -> None:
    super().__init__()
    self.device = torch.device("cuda")

  def load_my_model(self):
    self.tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH)

# 有一些预先设定参数
AUTHOR_PROMPT = "模仿："
TITLE_PROMPT = "作诗："
EOS_TOKEN = '</s>'

poem_model = PoemModel()
poem_model.load_my_model()
poem_model.model = poem_model.model.to('cuda')

MAX_AUTHOR_CHAR = 4
MAX_TITLE_CHAR = 12
MIN_CONTENT_CHAR = 10
MAX_CONTENT_CHAR = 64



# pingshui_json = './Pingshui_Rhyme.json'
# test = ['幽夢初成日，闲眠不掩门。晓窗风自落，夜枕雨相喧。','绝代佳人绝世姿，玉肌金骨照清池。不须更待妆梳洗，自是人间有别离。','秋风吹我衣，寒露洗我发。西风飒然至，吹我庭前叶。']
# with open(pingshui_json,'r',encoding='utf-8') as file:
#     yun_pingshui = json.load(file)

# pingsheng_dict = yun_pingshui['上平声部'].copy()
# pingsheng_dict.update(yun_pingshui['下平声部'])
# yun_dict = set()
# for r in pingsheng_dict:
#     for l in pingsheng_dict[r]:
#         yun_dict.update(l)
#         # print(l)

# print(yun_dict)
# class RhymeRewardProcessor(LogitsProcessor):
#     def __init__(self, input_ids, rhyme_dict, reward=2.0, penalty=-2.0):
#         self.rhyme_dict = rhyme_dict
#         self.reward = reward
#         self.penalty = penalty
#         self.generated_tokens = set(input_ids.tolist())

#     def __call__(self, input_ids, scores):
#         for i, seq in enumerate(input_ids):
#             last_token = seq[-1]  # 当前序列的最后一个 token
#             last_token = poem_model.tokenizer.decode(last_token)
#             if last_token in self.rhyme_dict:
#                 scores[i] += self.reward  # 符合押韵，奖励
#                 break
#             else:
#                 scores[i] += self.penalty  # 不符合押韵，惩罚
#         return scores

# # 定义 rhyme_processor
# rhyme_processor = RhymeRewardProcessor(rhyme_dict=yun_dict, reward=2.0, penalty=-2.0)


In [11]:
# 安装以下2个包方便文字处理和模型生成
# !pip install -q simplet5
# !pip install -q chinese-converter


def poem(title_str, opt_author=None, model=poem_model,
         is_input_traditional_chinese=False,
         num_beams=100):
  model.model = model.model.to('cuda')
  if opt_author:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR] + EOS_TOKEN + AUTHOR_PROMPT + opt_author[:MAX_AUTHOR_CHAR]
  else:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR]
  if is_input_traditional_chinese:
    in_request = chinese_converter.to_simplified(in_request)
  out = model.predict(in_request,
                      max_length=MAX_CONTENT_CHAR,
                      num_beams=num_beams)[0].replace(",", "，")
                      # logits_processor=LogitsProcessorList([rhyme_processor]))[0].replace(",", "，")

  
  if is_input_traditional_chinese:
    out = chinese_converter.to_traditional(out)
    print(f"標題： {in_request.replace('</s>', ' ')}\n詩歌： {out}")
  else:
    print(f"标题： {in_request.replace('</s>', ' ')}\n诗歌： {out}")


In [13]:
for title in ['秋思', '佳人', '相思',"幽梦"]:
  # Empty author means general style
  for author in ['', "杜甫", "李白", "李清照", "苏轼"]:
    poem(title, author)
  print()

标题： 作诗：秋思
诗歌： 梧桐叶落秋声早，蟋蟀声残夜气清。枕上忽惊新雁过，窗间已觉故人行。
标题： 作诗：秋思 模仿：杜甫
诗歌： 白露下百草，秋风吹我衣。天寒雁北至，日暮鸿南归。
标题： 作诗：秋思 模仿：李白
诗歌： 白露下百草，秋风吹我衣。西风入庭树，飒飒吹我衣。
标题： 作诗：秋思 模仿：李清照
诗歌： 梧桐叶落秋声早，蟋蟀声残夜气清。明月满庭人不寐，西风落叶满空庭。
标题： 作诗：秋思 模仿：苏轼
诗歌： 白露下百草，秋风吹我衣。西风入庭树，飒飒吹我衣。

标题： 作诗：佳人
诗歌： 绝代有佳人，天姿淑且真。肌肤冰雪莹，顔色雪霜匀。
标题： 作诗：佳人 模仿：杜甫
诗歌： 绝代有佳人，天姿绝世伦。肌肤冰雪莹，肌骨雪霜匀。
标题： 作诗：佳人 模仿：李白
诗歌： 绝代有佳人，超然绝世尘。倾国倾城色，倾国倾城春。
标题： 作诗：佳人 模仿：李清照
诗歌： 绝代佳人绝世姿，年年长向画堂期。不须更待妆梳洗，始信人间有别离。
标题： 作诗：佳人 模仿：苏轼
诗歌： 绝代有佳人，冰雪顔如玉。倾国倾城色，不与衆芳竞。

标题： 作诗：相思
诗歌： 美人在时花满枝，美人去后花满枝。愿得君心似杨柳，年年长向此时垂。
标题： 作诗：相思 模仿：杜甫
诗歌： 美人在时花满枝，美人去后花满枝。愿得君心似杨柳，年年长向此时垂。
标题： 作诗：相思 模仿：李白
诗歌： 美人在时花满枝，美人去后花满枝。我今归去不复返，妾亦未归君不知。
标题： 作诗：相思 模仿：李清照
诗歌： 梧桐叶落秋风老，蟋蟀声中秋夜长。织女机边裁白纻，姮娥镜裏弄青黄。
标题： 作诗：相思 模仿：苏轼
诗歌： 美人在时花满枝，美人去后花满枝。我今欲寄相思字，爲君题作相思诗。

标题： 作诗：幽梦
诗歌： 幽夢初疑是夢中，恍疑身在广寒宫。蓬莱咫尺无由到，却向人间作夢中。
标题： 作诗：幽梦 模仿：杜甫
诗歌： 幽夢初疑是夢中，觉来还似夢中同。桃花落尽春犹在，柳絮飞时雨未通。
标题： 作诗：幽梦 模仿：李白
诗歌： 幽夢不可得，百年如转蓬。不知身是夢，但觉夢非熊。
标题： 作诗：幽梦 模仿：李清照
诗歌： 幽夢初从枕上回，觉来浑似夢中来。却疑身是辽天鹤，飞到蓬莱第几台
标题： 作诗：幽梦 模仿：苏轼
诗歌： 幽夢初从枕上回，觉来浑似夢中来。平生不作功名想，老去空惊岁月催。

